In [1]:
import os
import numpy as np
from tqdm import tqdm

from sqa_gen.data_extraction import extract_data_from_file, data_split_tools, series2graph

import torch

In [2]:
data_folder = 'dataset/opportunity'
file_list = [ i for i in os.listdir(data_folder) if '.dat' in i]

In [3]:
file_list

['S3-ADL4.dat',
 'S3-ADL1.dat',
 'S1-ADL5.dat',
 'S1-ADL4.dat',
 'S4-ADL5.dat',
 'S2-ADL4.dat',
 'S3-Drill.dat',
 'S2-ADL1.dat',
 'S1-ADL1.dat',
 'S4-ADL2.dat',
 'S2-ADL2.dat',
 'S4-ADL3.dat',
 'S2-ADL3.dat',
 'S3-ADL2.dat',
 'S1-ADL2.dat',
 'S4-ADL4.dat',
 'S4-Drill.dat',
 'S3-ADL5.dat',
 'S4-ADL1.dat',
 'S3-ADL3.dat',
 'S2-Drill.dat',
 'S1-ADL3.dat',
 'S2-ADL5.dat',
 'S1-Drill.dat']

In [4]:
label1 = np.zeros([0,1])
label2 = np.zeros([0,1])
data = np.zeros([0,77])

datapath = 'dataset/opportunity'

for file_i in file_list:
    print(file_i)
    label_y, label_list, data_x = extract_data_from_file(file_i, datapath,
                                                     plot_option = False, 
                                                     show_other = False)
    label1 = np.concatenate((label1, label_y[0].reshape([-1, 1])), axis = 0)
    label2 = np.concatenate((label2, label_y[1].reshape([-1, 1])), axis = 0)

    data = np.concatenate((data, data_x), axis = 0)

S3-ADL4.dat
S3-ADL1.dat
S1-ADL5.dat
S1-ADL4.dat
S4-ADL5.dat
S2-ADL4.dat
S3-Drill.dat
S2-ADL1.dat
S1-ADL1.dat
S4-ADL2.dat
S2-ADL2.dat
S4-ADL3.dat
S2-ADL3.dat
S3-ADL2.dat
S1-ADL2.dat
S4-ADL4.dat
S4-Drill.dat
S3-ADL5.dat
S4-ADL1.dat
S3-ADL3.dat
S2-Drill.dat
S1-ADL3.dat
S2-ADL5.dat
S1-Drill.dat


# Extract data

In [5]:
label_y, label_list, data_x = extract_data_from_file('S1-ADL5.dat', datapath,
                                                     plot_option = False, 
                                                     show_other = False)

print('=============label_y=============')
print(np.array(label_y).shape)  # (3, 30048) -> 3: category / 30048: number of data(time-stemp without NaN)
print(label_y)  # [0]: 1~18 / [1]: 1~5 / [2]: 1~6 -> classes
print('=======================================')
print('=============label_list=============')
print(len(label_list))  # category
print(len(label_list[0]))   # mid
print(len(label_list[1]))   # loc
print(len(label_list[2]))   # high
print(label_list)   # class name
print('=======================================')
print('=============data_x=============')
print(data_x.shape) # (30048, 77)   # (#data(time-stemp), channel)
print('=======================================')

=============label_y=============
(3, 30048)
[array([1, 1, 1, ..., 1, 1, 1]), array([1, 1, 1, ..., 1, 1, 1]), array([1, 1, 1, ..., 1, 1, 1])]
=============label_list=============
3
18
5
6
[['Other', 'Open Door 1', 'Open Door 2', 'Close Door 1', 'Close Door 2', 'Open Fridge', 'Close Fridge', 'Open Dishwasher', 'Close Dishwasher', 'Open Drawer 1', 'Close Drawer 1', 'Open Drawer 2', 'Close Drawer 2', 'Open Drawer 3', 'Close Drawer 3', 'Clean Table', 'Drink from Cup', 'Toggle Switch'], ['Other', 'Stand', 'Walk', 'Sit', 'Lie'], ['Other', 'Relaxing', 'Coffee time', 'Early morning', 'Cleanup', 'Sandwich time']]
=============data_x=============
(30048, 77)


## Label List

In [6]:
label_list[0][0] = 'Do other activities'
label_list[0][1] = 'Open the front Door'
label_list[0][2] = 'Open the back Door'
label_list[0][3] = 'Close the front Door'
label_list[0][4] = 'Close the back Door'
label_list[0][5] = 'Open the Fridge'
label_list[0][6] = 'Close the Fridge'
label_list[0][7] = 'Open the Dishwasher'
label_list[0][8] = 'Close the Dishwasher'
label_list[0][9] = 'Open the first Drawer'
label_list[0][10] = 'Close the first Drawer'
label_list[0][11] = 'Open the second Drawer'
label_list[0][12] = 'Close the second Drawer'
label_list[0][13] = 'Open the third Drawer'
label_list[0][14] = 'Close the third Drawer'
label_list[0][15] = 'Clean the Table'
label_list[0][16] = 'Drink from the Cup'
label_list[0][17] = 'Toggle the Switch'

## Split data

In [7]:
window_size = 1500
stride = 400
device = 0
torch.cuda.set_device(device)

for startpoint_i in tqdm(range(0, data_x.shape[0]-window_size, stride)):
    # the sampling rate for opportunity is 30HZ, window is 60s
    seg_x ,seg_y_list, startpoint = data_split_tools(data_x, 
                                                        label_y, 
                                                        window_size,
                                                        startpoint = startpoint_i)
    
    seg_x = np.expand_dims(seg_x, axis=1)
    seg_x = np.expand_dims(seg_x, axis=1)
    seg_x = torch.tensor(seg_x, dtype=torch.float32).to(device)

    print('=============seg_x shape=============')
    print(seg_x.shape)  # reshape input data (windowsize, 1, 1, channel)
    print('=======================================')
    print('=============seg_y_liste=============')
    print(np.array(seg_y_list).shape)   # (3, windowsize) / 3=[mid, loc, high]
    print('=======================================')
    print('=============startpoint=============')
    print(startpoint)  # reshape input data (windowsize, 1, 1, channel)
    print('=======================================')


100%|██████████| 72/72 [00:00<00:00, 105.45it/s]

=============seg_x shape=============
torch.Size([1500, 1, 1, 77])
=============seg_y_liste=============
(3, 1500)
=============startpoint=============
0
=============seg_x shape=============
torch.Size([1500, 1, 1, 77])
=============seg_y_liste=============
(3, 1500)
=============startpoint=============
400
=============seg_x shape=============
torch.Size([1500, 1, 1, 77])
=============seg_y_liste=============
(3, 1500)
=============startpoint=============
800
=============seg_x shape=============
torch.Size([1500, 1, 1, 77])
=============seg_y_liste=============
(3, 1500)
=============startpoint=============
1200
=============seg_x shape=============
torch.Size([1500, 1, 1, 77])
=============seg_y_liste=============
(3, 1500)
=============startpoint=============
1600
=============seg_x shape=============
torch.Size([1500, 1, 1, 77])
=============seg_y_liste=============
(3, 1500)
=============startpoint=============
2000
=============seg_x shape=============
torch.Size([1500, 1, 1, 7

## series2graph

In [8]:
show_other = False

for startpoint_i in tqdm(range(0, data_x.shape[0]-window_size, stride)):
    # the sampling rate for opportunity is 30HZ, window is 60s
    seg_x ,seg_y_list, startpoint = data_split_tools(data_x, 
                                                        label_y, 
                                                        window_size,
                                                        startpoint = startpoint_i)

    scene_list_1 = series2graph(seg_y_list[0], label_list[0], show_graph = True, show_other = show_other)
    scene_list_2 = series2graph(seg_y_list[1], label_list[1], show_graph = True)
    scene_lists = [scene_list_1, scene_list_2]

    print('=============seg_x=============')
    print(seg_y_list[0])
    print(label_list[0])
    print('=======================================')
    print(scene_list_1)
    print(scene_list_2)
    print('=======================================')

100%|██████████| 72/72 [00:00<00:00, 1353.96it/s]

END [60]


[42.8]Stand 0.6==>
[43.5]Walk 3.1==>
END [60]


=============seg_x=============
[1 1 1 ... 1 1 1]
['Do other activities', 'Open the front Door', 'Open the back Door', 'Close the front Door', 'Close the back Door', 'Open the Fridge', 'Close the Fridge', 'Open the Dishwasher', 'Close the Dishwasher', 'Open the first Drawer', 'Close the first Drawer', 'Open the second Drawer', 'Close the second Drawer', 'Open the third Drawer', 'Close the third Drawer', 'Clean the Table', 'Drink from the Cup', 'Toggle the Switch']
[array([], dtype=int64), [], [], []]
[array([0, 1]), [2, 3], [0.5999999999999943, 3.1000000000000014], [42.833333333333336, 43.53333333333333]]
END [60]


[29.5]Stand 0.6==>
[30.2]Walk 3.1==>
[38.5]Lie 11.47==>
END [60]


=============seg_x=============
[1 1 1 ... 1 1 1]
['Do other activities', 'Open the front Door', 'Open the back Door', 'Close the front Door', 'Close the back Door', 'Open the Fridge', 'Close the Fridge', 'Open the Dishwasher', 'Close the Dishwasher'

## Get predictions from the models

In [9]:
from models.ConvLSTM import ConvLSTM

save_model_folder = 'trained_models/'
model_name = 'single_1'
save_path1 = save_model_folder+ 'opp_model/'+ model_name+'.pt'
model_name = 'single_2'
save_path2 = save_model_folder + 'opp_model/'+ model_name+'.pt'

dim = 77
win_len = 1
feature = 64
drop_rate = 0.3
num_class1 = 18
num_class2 = 5

if os.path.isfile(save_path1) and os.path.isfile(save_path1):
    print('Load models')
    trained_model_1 = ConvLSTM(dim=dim, win_len=win_len, num_classes_1=num_class1, num_feat_map=feature, dropout_rate=drop_rate)
    trained_model_1.load_state_dict(torch.load(save_path1))

    trained_model_2 = ConvLSTM(dim=dim, win_len=win_len, num_classes_1=num_class2, num_feat_map=feature, dropout_rate=drop_rate)
    trained_model_2.load_state_dict(torch.load(save_path2))

Load models


In [10]:
trained_model_1 = trained_model_1.to(device)
trained_model_2 = trained_model_2.to(device)
trained_model_1.eval()
trained_model_2.eval()

seg_x = np.expand_dims(seg_x, axis=1)
seg_x = np.expand_dims(seg_x, axis=1)
seg_x = torch.tensor(seg_x, dtype=torch.float32).to(device)

with torch.no_grad():  # Ensure no gradients are calculated
    # Get predictions from the models
    seg_y1_pred = trained_model_1(seg_x)
    seg_y2_pred = trained_model_2(seg_x)
    seg_y1_pred = torch.argmax(seg_y1_pred, dim=1)
    seg_y2_pred = torch.argmax(seg_y2_pred, dim=1)

    seg_y_list_pred = [seg_y1_pred, seg_y2_pred]

    scene_list_1_pred = series2graph(seg_y_list_pred[0], label_list[0], show_graph = False)
    scene_list_2_pred = series2graph(seg_y_list_pred[1], label_list[1], show_graph = False)

    scene_lists_pred = [scene_list_1_pred, scene_list_2_pred]

    print('=======================================')
    print(scene_list_1_pred)
    print(scene_list_2_pred)
    print('=======================================')

[array([0]), [tensor(0, device='cuda:0')], [50.0], [0.0]]
[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]), [tensor(4, device='cuda:0'), tensor(0, device='cuda:0'), tensor(4, device='cuda:0'), tensor(0, device='cuda:0'), tensor(4, device='cuda:0'), tensor(0, device='cuda:0'), tensor(0, device='cuda:0'), tensor(0, device='cuda:0'), tensor(0, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(0, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(0, device='cuda:0'), tensor(2, device='cuda:0'), tensor(0, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(2, device='cuda:0'), tensor(0, device='cuda:0'), tensor(0, device='cuda:0'), tensor(0, device='cuda:0'), tensor(0, device='cuda:0'), tensor(0, device='cuda:0'

## Question Generator

In [11]:
import json
import random
import numpy as np
from pattern import en
from sqa_gen.functional_program import *

# Alternative approach: specify the tense (https://www.clips.uantwerpen.be/pages/pattern-en)
def fixSentenceTense(question, actions, relations, combinators):
    
    text = question[0]
    actionCount = 1

    print('*'*10)
    print(question)
    print(text)
    print(actions)
    print('*'*10)
    
    #Replace the actions
    for action in actions:
        # verb should be first word of action
        # swd: if there's only one word, then handle differently
        action_split = action.split(' ')
        verb = action_split[0] 
        # get tense:
        verb = en.conjugate(verb, tense=question[actionCount][0], person=question[actionCount][1], number=question[actionCount][2])
        replacedAction = '<A'+str(actionCount)+'>'
        if len(action_split)== 1:
            text = text.replace(
                text[text.find(replacedAction):(text.find(replacedAction)+len(replacedAction))], 
                                                   verb)
        else:
            text = text.replace(
                text[text.find(replacedAction):(text.find(replacedAction)+len(replacedAction))], 
                                                   verb + action[action.find(' '):])
        actionCount = actionCount + 1
    
    #Replace the relations
    relationCount = 1
    for relation in relations:
        replacedRelation = '[R'+str(relationCount)+']'
#         print("Before replacement ("+replacedRelation+"): ", text)
        text = text.replace(text[text.find(replacedRelation):(text.find(replacedRelation)+len(replacedRelation))], 
                                               relation)
#         print("After replacement: ",text)
        relationCount = relationCount + 1
    
    #Replace the combinators
    combinatorCount = 1
    for combinator in combinators:
        replacedCombinator = '[C'+str(combinatorCount)+']'
        text = text.replace(text[text.find(replacedCombinator):(text.find(replacedCombinator)+len(replacedCombinator))], 
                                               combinator)
        combinatorCount = combinatorCount + 1
        
    return text

def question_generator(scene_lists, scene_lists_pred,
                       question_family_file, 
                       label_list,
                       show_other = False,
                       question_validation = True,
                       diagnose = False 
                  ):
    
    """
    Generate questions and answers in NLP form.
    Input: 
    scene_lists: the semantic representation of sensory context data. (2 lvls)
    scene_lists_pred: the predicted semantic representation of sensory context.
    question_family_file: the filename of question family (JSON format)
    label_list: the corresponding label_list for the annotations in scene_lists.
    Output:
    """
    
    def question_summary(q_counter, q_all_num, q_id):
        print('===================================')
        print('Generate %d/%d questions in total of question family %d.'%(q_counter, q_all_num, q_id))
        return
    
    # loading question family from JSON
    with open(question_family_file) as json_file:
        question_family = json.load(json_file)
    
    # diagnose print
    if diagnose:
        print('===================================')
        print('Starting generating questions: ')
        print('Validation : ', question_validation)
        print('Size of question families: ', len(question_family['questions'])
        
    # dict for change binary answer to natural language
    answer_dict = {'True': 'Yes', 'False': 'No'}
    
    # all possible relations and logics
    relation_family = ['Before', 'After', 'Preceding', 'Following']
    logic_combinator_family = ['AND', 'OR']
    
    # all involvded actions and locomotions  # changed to all possible actions
    # whether using "other" as one of the activity: yes... change 2 places
    if show_other:
        unique_actions = np.array(range(1, 19))# in total 18 classes, 1-18
        unique_loc = np.array(range(1,6))
    else:
        unique_actions = np.array(range(2, 19)) # the 1st class is other
        unique_loc = np.array(range(2,6))
        
#     unique_actions = np.unique(scene_lists[0][1])
#     unique_loc = np.unique(scene_lists[1][1])



    # diagnose print
    if diagnose:
        print('\n===================================')
        print('Unique actions: ')
        print(unique_actions)
        print('Unique locomotions: ')
        print(unique_loc)
        
    
    # initialize question/ans/q_tpye lists:
    question_family_index = []
    question_nl = []
    answer_nl = []
    answer_nl_p = [] # store the predicted answer from Neural-symbolic appraoch
    question_struct = []  # store the structure of question. In the form of a string: ID_act1_act2_combine_relate1_relate2
    
    # generating questions: if using an automatic approach, need to use recursion. 
    
    # =========================== question 0 ===========================
    q_id = 0
    q_counter = 0
    q_all_num = len(unique_actions)**1 * len(relation_family)**0 * len(logic_combinator_family)**0 

    for action_1 in unique_actions:
        actions = [label_list[0][int(action_1)-1]]
        
        
        q_f_nlp = question_family['questions'][q_id]['texts']
        question_id = random.randint(0,len(q_f_nlp)-1)
        question_nlp = q_f_nlp[question_id]
        
        question_nlp = fixSentenceTense(question_nlp, actions,[],[])
            #question_nlp.replace('<A1>', action_1_nlp)

        # Try generating questions for all possible combinations
        try:         
            ans_sm = str(function_families[q_id](action_1 , scene_lists))
            ans_nlp = answer_dict[ans_sm]

    #         print(question_nlp, ans_nlp)
            question_family_index.append(q_id)
            question_nl.append(question_nlp)
            answer_nl.append(ans_nlp)
            question_struct.append(str(q_id)+'_'+str(actions))
            
            # Try generating answers for the question
            try:
                ans_sm_p = str(function_families[q_id](action_1 , scene_lists_pred, valid_ext = True))
                ans_nlp_p = answer_dict[ans_sm_p]
            except ValueError:
                ans_nlp_p = 'Invalid'
            answer_nl_p.append(ans_nlp_p)

            q_counter += 1
        except ValueError:  ####### The question is not valid in the context
            pass
        

    if diagnose:
        question_summary(q_counter, q_all_num, q_id)


    # =========================== question 1 ===========================
    q_id = 1
    q_counter = 0
    q_all_num = len(unique_actions)**1 * len(relation_family)**0 * len(logic_combinator_family)**0 

    for action_1 in unique_actions:
        actions = [label_list[0][int(action_1)-1]]

        q_f_nlp = question_family['questions'][q_id]['texts']
        question_id = random.randint(0,len(q_f_nlp)-1)
        question_nlp = q_f_nlp[question_id]
        #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
        question_nlp = fixSentenceTense(question_nlp, actions,[],[])
        
        # Try generating questions for all possible combinations
        try:         
            ans_sm = str(function_families[q_id](action_1 , scene_lists))
            ans_nlp = ans_sm

            #         print(question_nlp, ans_nlp)
            question_family_index.append(q_id)
            question_nl.append(question_nlp)
            answer_nl.append(ans_nlp)
            question_struct.append(str(q_id)+'_'+str(actions))
            
            # Try generating answers for the question
            try:
                ans_sm_p = str(function_families[q_id](action_1 , scene_lists_pred, valid_ext = True))
                ans_nlp_p = ans_sm_p
            except ValueError:
                ans_nlp_p = 'Invalid'
            answer_nl_p.append(ans_nlp_p)

            q_counter += 1
        except ValueError:  ####### The question is not valid in the context
            pass  

    if diagnose:
        question_summary(q_counter, q_all_num, q_id)


    # =========================== question 2 ===========================
    # for question 2
    q_id = 2
    q_counter = 0
    q_all_num = len(unique_actions)**2 * len(relation_family)**1 * len(logic_combinator_family)**0 

    for relation in relation_family:
        for action_1 in unique_actions:
            for action_2 in unique_actions:
                if action_2 == action_1: # avoid meaningless questions
                    continue

                actions = [label_list[0][int(action_1)-1],label_list[0][int(action_2)-1]]
                #action_2_nlp = label_list[0][int(action_2)-1]

                q_f_nlp = question_family['questions'][q_id]['texts']
                question_id = random.randint(0,len(q_f_nlp)-1)
                question_nlp = q_f_nlp[question_id]
                question_nlp = fixSentenceTense(question_nlp, actions,[relation],[])
                #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
                #question_nlp = question_nlp.replace('<A2>', action_2_nlp)
                #question_nlp = question_nlp.replace('[R1]', relation)

                # Try generating questions for all possible combinations
                try:         
                    ans_sm = str(function_families[q_id](action_1, action_2, 
                                                          relation, 
                                                          scene_lists,
                                                          question_validation))
                    ans_nlp = answer_dict[ans_sm]

            #         print(question_nlp, ans_nlp)
                    question_family_index.append(q_id)
                    question_nl.append(question_nlp)
                    answer_nl.append(ans_nlp)
                    question_struct.append(str(q_id)+'_'+str(actions) + '_' + relation)
                    
                    # Try generating answers for the question
                    try:
                        ans_sm_p = str(function_families[q_id](action_1, action_2, 
                                                          relation, 
                                                          scene_lists_pred,
                                                          question_validation = False, valid_ext = True))
                        ans_nlp_p = answer_dict[ans_sm_p]
                    except ValueError:
                        ans_nlp_p = 'Invalid'
                    answer_nl_p.append(ans_nlp_p)

                    q_counter += 1
                except ValueError:  ####### The question is not valid in the context
                    pass

    if diagnose:
        question_summary(q_counter, q_all_num, q_id)

    # =========================== question 3 ===========================
    # for question 3
    q_id = 3
    q_counter = 0
    q_all_num = len(unique_actions)**3 * len(relation_family)**2 * len(logic_combinator_family)**1 

    for c_logic_1 in logic_combinator_family:
        for relation_1 in relation_family:
            for relation_2 in relation_family:
                for action_1 in unique_actions:
                    for action_2 in unique_actions:
                        if action_2 == action_1: # avoid meaningless questions
                            continue
                        for action_3 in unique_actions:
                            if action_3 == action_1: # avoid meaningless questions
                                continue
                            if action_2 == action_3 and relation_1 == relation_2: # avoid meaningless questions
                                continue
                            

                            actions =  [label_list[0][int(action_1)-1], label_list[0][int(action_2)-1], label_list[0][int(action_3)-1]]
                            relations = [relation_1, relation_2]
                            combinators = [c_logic_1]
                            #action_1_nlp = label_list[0][int(action_1)-1]
                            #action_2_nlp = label_list[0][int(action_2)-1]
                            #action_3_nlp = label_list[0][int(action_3)-1]

                            q_f_nlp = question_family['questions'][q_id]['texts']
                            question_id = random.randint(0,len(q_f_nlp)-1)
                            question_nlp = q_f_nlp[question_id]
                            question_nlp = fixSentenceTense(question_nlp, actions, relations, combinators)
                            #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
                            #question_nlp = question_nlp.replace('<A2>', action_2_nlp)
                            #question_nlp = question_nlp.replace('<A3>', action_3_nlp)
                            #question_nlp = question_nlp.replace('[R1]', relation_1)
                            #question_nlp = question_nlp.replace('[R2]', relation_2)
                            #question_nlp = question_nlp.replace('[C1]', c_logic_1)

                            # Try generating questions for all possible combinations
                            try:
                                ans_sm = function_families[q_id](action_1, action_2, action_3, 
                                                                 relation_1, relation_2, 
                                                                 c_logic_1, 
                                                                 scene_lists,
                                                                 question_validation) 
                                ans_sm = str(ans_sm)
                                ans_nlp = answer_dict[ans_sm]

                        #         print(question_nlp, ans_nlp)
                                question_family_index.append(q_id)
                                question_nl.append(question_nlp)
                                answer_nl.append(ans_nlp)
                                question_struct.append(str(q_id)+'_'+str(actions) + '_' + str(relations)+ '_'+str(combinators))

                                # Try generating answers for the question
                                try:
                                    ans_sm_p = function_families[q_id](action_1, action_2, action_3, 
                                                                 relation_1, relation_2, 
                                                                 c_logic_1, 
                                                                 scene_lists_pred,
                                                                 question_validation = False, valid_ext = True)
                                    ans_nlp_p = answer_dict[str(ans_sm_p)]
                                except ValueError:
                                    ans_nlp_p = 'Invalid'
                                answer_nl_p.append(ans_nlp_p)
                                
                                q_counter += 1
                            except ValueError:  ####### The question is not valid in the context
                                pass   


    if diagnose:
        question_summary(q_counter, q_all_num, q_id)



    # =========================== question 4 ===========================
    # for question 4
    q_id = 4
    q_counter = 0
    q_all_num = len(unique_actions)**2 * len(relation_family)**1 * len(logic_combinator_family)**0 

    for relation_1 in relation_family:
        for action_1 in unique_actions:
            for action_2 in unique_actions:
                if action_1 == action_2: # avoid meaningless questions
                    continue

                actions = [label_list[0][int(action_1)-1],label_list[0][int(action_2)-1]]
                relations = [relation_1]
                #action_1_nlp = label_list[0][int(action_1)-1]
                #action_2_nlp = label_list[0][int(action_2)-1]

                q_f_nlp = question_family['questions'][q_id]['texts']
                question_id = random.randint(0,len(q_f_nlp)-1)
                question_nlp = q_f_nlp[question_id]

                question_nlp = fixSentenceTense(question_nlp, actions, relations, [])
                #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
                #question_nlp = question_nlp.replace('<A2>', action_2_nlp)
                #question_nlp = question_nlp.replace('[R1]', relation_1)

                # Try generating questions for all possible combinations
                try:
                    ans_sm = function_families[q_id](action_1, action_2, 
                                                     relation_1, 
                                                     scene_lists,
                                                     question_validation) 
                    ans_sm = str(ans_sm)
    #                 ans_nlp = answer_dict[ans_sm]
                    ans_nlp = ans_sm

            #         print(question_nlp, ans_nlp)
                    question_family_index.append(q_id)
                    question_nl.append(question_nlp)
                    answer_nl.append(ans_nlp)
                    question_struct.append(str(q_id)+'_'+str(actions) + '_' + str(relations))
                    
                    # Try generating answers for the question
                    try:
                        ans_sm_p = function_families[q_id](action_1, action_2, 
                                                     relation_1, 
                                                     scene_lists_pred,
                                                     question_validation = False, valid_ext = True)
                        ans_nlp_p = str(ans_sm_p)
                    except ValueError:
                        ans_nlp_p = 'Invalid'
                    answer_nl_p.append(ans_nlp_p)
                                

                    q_counter += 1
                except ValueError:  ####### The question is not valid in the context
                    pass


    if diagnose:
        question_summary(q_counter, q_all_num, q_id)




    # =========================== question 5 ===========================
    # for question 5
    q_id = 5
    q_counter = 0
    q_all_num = len(unique_actions)**3 * len(relation_family)**2 * len(logic_combinator_family)**1 

    for c_logic_1 in logic_combinator_family:
        for relation_1 in relation_family:
            for relation_2 in relation_family:
                for action_1 in unique_actions:
                    for action_2 in unique_actions:
                        if action_1 == action_2: # avoid meaningless questions
                            continue
                        for action_3 in unique_actions:
                            if action_1 == action_3: # avoid meaningless questions
                                continue
                            if action_2 == action_3 and relation_1 == relation_2: # avoid meaningless questions
                                continue
                            

                            actions = [label_list[0][int(action_1)-1], label_list[0][int(action_2)-1], label_list[0][int(action_3)-1]]
                            relations = [relation_1, relation_2]
                            combinators = [c_logic_1]
                            #action_1_nlp = label_list[0][int(action_1)-1]
                            #action_2_nlp = label_list[0][int(action_2)-1]
                            #action_3_nlp = label_list[0][int(action_3)-1]

                            q_f_nlp = question_family['questions'][q_id]['texts']
                            question_id = random.randint(0,len(q_f_nlp)-1)
                            question_nlp = q_f_nlp[question_id]
                            
                            question_nlp = fixSentenceTense(question_nlp, actions, relations, combinators) 
                            #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
                            #question_nlp = question_nlp.replace('<A2>', action_2_nlp)
                            #question_nlp = question_nlp.replace('<A3>', action_3_nlp)
                            #question_nlp = question_nlp.replace('[R1]', relation_1)
                            #question_nlp = question_nlp.replace('[R2]', relation_2)
                            #question_nlp = question_nlp.replace('[C1]', c_logic_1)

                            # Try generating questions for all possible combinations
                            try:
                                ans_sm = function_families[q_id](action_1, action_2, action_3, 
                                                                 relation_1, relation_2, 
                                                                 c_logic_1, 
                                                                 scene_lists,
                                                                 question_validation) 
                                ans_sm = str(ans_sm)
                #                 ans_nlp = answer_dict[ans_sm]
                                ans_nlp = ans_sm

                                        #         print(question_nlp, ans_nlp)
                                question_family_index.append(q_id)
                                question_nl.append(question_nlp)
                                answer_nl.append(ans_nlp)
                                question_struct.append(str(q_id)+'_'+str(actions) + '_' + str(relations)+ '_'+str(combinators))

                                # Try generating answers for the question
                                try:
                                    ans_sm_p = function_families[q_id](action_1, action_2, action_3, 
                                                                 relation_1, relation_2, 
                                                                 c_logic_1, 
                                                                 scene_lists_pred,
                                                                 question_validation = False, valid_ext = True) 
                                    ans_nlp_p = str(ans_sm_p)
                                except ValueError:
                                    ans_nlp_p = 'Invalid'
                                answer_nl_p.append(ans_nlp_p)
                                
                                q_counter += 1
                            except ValueError:  ####### The question is not valid in the context
                                pass


    if diagnose:
        question_summary(q_counter, q_all_num, q_id) 




    # =========================== question 6 ===========================
    # for question 6
    q_id = 6
    q_counter = 0
    q_all_num = len(unique_actions)**1 * len(relation_family)**1 * len(logic_combinator_family)**0 

    for relation_1 in relation_family:
        for action_1 in unique_actions:

            #action_1_nlp = label_list[0][int(action_1)-1]
            actions = [label_list[0][int(action_1)-1]]
            relations = [relation_1]

            q_f_nlp = question_family['questions'][q_id]['texts']
            question_id = random.randint(0,len(q_f_nlp)-1)
            question_nlp = q_f_nlp[question_id]

            
            question_nlp = fixSentenceTense(question_nlp, actions, relations, []) 
            #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
            #question_nlp = question_nlp.replace('[R1]', relation_1)

            # Try generating questions for all possible combinations
            try:
                ans_sm = function_families[q_id](action_1,
                                                 relation_1, 
                                                 scene_lists)
    #             ans_sm = str(ans_sm)
        #                 ans_nlp = answer_dict[ans_sm]
                ans_nlp = label_list[0][int(ans_sm)-1]

            #         print(question_nlp, ans_nlp)
                question_family_index.append(q_id)
                question_nl.append(question_nlp)
                answer_nl.append(ans_nlp)
                question_struct.append(str(q_id)+'_'+str(actions) + '_' + str(relations))
                
                # Try generating answers for the question
                try:
                    ans_sm_p = function_families[q_id](action_1,
                                                 relation_1, 
                                                 scene_lists_pred , valid_ext = True)
                    ans_nlp_p = label_list[0][int(ans_sm_p)-1]
                except ValueError:
                    ans_nlp_p = 'Invalid'
                answer_nl_p.append(ans_nlp_p)

                q_counter += 1
            except ValueError:  ####### The question is not valid in the context
                pass


    if diagnose:
        question_summary(q_counter, q_all_num, q_id)


    # =========================== question 7 ===========================
    # for question 7
    q_id = 7
    q_counter = 0
    q_all_num = len(unique_actions)**2 * len(relation_family)**2 * len(logic_combinator_family)**0

    for relation_1 in relation_family:
        for relation_2 in relation_family:
            for action_1 in unique_actions:
                for action_2 in unique_actions:
                    if action_2 == action_1: # avoid meaningless questions
                        continue
                    

                    actions = [label_list[0][int(action_1)-1],label_list[0][int(action_2)-1]] 
                    #action_1_nlp = label_list[0][int(action_1)-1]
                    #action_2_nlp = label_list[0][int(action_2)-1]
                    relations = [relation_1, relation_2]

                    q_f_nlp = question_family['questions'][q_id]['texts']
                    question_id = random.randint(0,len(q_f_nlp)-1)
                    question_nlp = q_f_nlp[question_id]
                    
                    question_nlp = fixSentenceTense(question_nlp, actions, relations, []) 
                    #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
                    #question_nlp = question_nlp.replace('<A2>', action_2_nlp)
                    #question_nlp = question_nlp.replace('[R1]', relation_1)
                    #question_nlp = question_nlp.replace('[R2]', relation_2)

                    # Try generating questions for all possible combinations
                    try:
                        ans_sm = function_families[q_id](action_1, action_2, 
                                                         relation_1, relation_2, 
                                                         scene_lists,
                                                         question_validation) 

        #                 ans_nlp = answer_dict[ans_sm]
                        ans_nlp = label_list[0][int(ans_sm)-1]

                #         print(question_nlp, ans_nlp)
                        question_family_index.append(q_id)
                        question_nl.append(question_nlp)
                        answer_nl.append(ans_nlp)
                        question_struct.append(str(q_id)+'_'+str(actions) + '_' + str(relations))
                        
                        # Try generating answers for the question
                        try:
                            ans_sm_p = function_families[q_id](action_1, action_2, 
                                                         relation_1, relation_2, 
                                                         scene_lists_pred,
                                                         question_validation = False, valid_ext = True) 
                            ans_nlp_p = label_list[0][int(ans_sm_p)-1]
                        except ValueError:
                            ans_nlp_p = 'Invalid'
                        answer_nl_p.append(ans_nlp_p)

                        q_counter += 1
                    except ValueError:  ####### The question is not valid in the context
                        pass


    if diagnose:
        question_summary(q_counter, q_all_num, q_id)


    # =========================== question 8 ===========================
    # for question 8
    q_id = 8
    q_counter = 0
    q_all_num = len(unique_actions)**1 * len(relation_family)**0 * len(logic_combinator_family)**0

    for action_1 in unique_actions:
        #action_1_nlp = label_list[0][int(action_1)-1]
        actions = [label_list[0][int(action_1)-1]]
        
        q_f_nlp = question_family['questions'][q_id]['texts']
        question_id = random.randint(0,len(q_f_nlp)-1)
        question_nlp = q_f_nlp[question_id]

        question_nlp = fixSentenceTense(question_nlp, actions, [], []) 
        #question_nlp = question_nlp.replace('<A1>', action_1_nlp)

        # Try generating questions for all possible combinations
        try: 
            ans_sm = function_families[q_id](action_1, 
                                             scene_lists)
            ans_sm = str(ans_sm)
            ans_nlp = answer_dict[ans_sm]

    #         print(question_nlp, ans_nlp)
            question_family_index.append(q_id)
            question_nl.append(question_nlp)
            answer_nl.append(ans_nlp)
            question_struct.append(str(q_id)+'_'+str(actions))
            
            # Try generating answers for the question
            try:
                ans_sm_p = function_families[q_id](action_1, 
                                             scene_lists_pred, valid_ext = True)
                ans_nlp_p = answer_dict[str(ans_sm_p)]
            except ValueError:
                ans_nlp_p = 'Invalid'
            answer_nl_p.append(ans_nlp_p)

            q_counter += 1
        except ValueError:  ####### The question is not valid in the context
            pass   

    if diagnose:
        question_summary(q_counter, q_all_num, q_id)



    # =========================== question 9 ===========================
    # for question 9
    q_id = 9
    q_counter = 0
    q_all_num = len(unique_actions)**2 * len(relation_family)**2 * len(logic_combinator_family)**0

    for relation_1 in relation_family:
        for relation_2 in relation_family:
            for action_1 in unique_actions:
                for action_2 in unique_actions:
                    if action_1 == action_2:
                        continue

                    actions = [label_list[0][int(action_1)-1],label_list[0][int(action_2)-1]]
                    #action_1_nlp = label_list[0][int(action_1)-1]
                    #action_2_nlp = label_list[0][int(action_2)-1]
                    relations = [relation_1, relation_2]

                    q_f_nlp = question_family['questions'][q_id]['texts']
                    question_id = random.randint(0,len(q_f_nlp)-1)
                    question_nlp = q_f_nlp[question_id]
                    question_nlp = fixSentenceTense(question_nlp, actions, relations, []) 

                    #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
                    #question_nlp = question_nlp.replace('<A2>', action_2_nlp)
                    #question_nlp = question_nlp.replace('[R1]', relation_1)
                    #question_nlp = question_nlp.replace('[R2]', relation_2)

                    # Try generating questions for all possible combinations
                    try:
                        ans_sm = function_families[q_id](action_1, action_2, 
                                                         relation_1, relation_2, 
                                                         scene_lists)

                        ans_sm = str(ans_sm)
                        ans_nlp = answer_dict[ans_sm]

                #         print(question_nlp, ans_nlp)
                        question_family_index.append(q_id)
                        question_nl.append(question_nlp)
                        answer_nl.append(ans_nlp)
                        question_struct.append(str(q_id)+'_'+str(actions) + '_' + str(relations))
                        
                        # Try generating answers for the question
                        try:
                            ans_sm_p = function_families[q_id](action_1, action_2, 
                                                         relation_1, relation_2, 
                                                         scene_lists_pred, valid_ext = True)
                            ans_nlp_p = answer_dict[str(ans_sm_p)]
                        except ValueError:
                            ans_nlp_p = 'Invalid'
                        answer_nl_p.append(ans_nlp_p)

                        q_counter += 1
                    except ValueError:  ####### The question is not valid in the context
                        pass


    if diagnose:
        question_summary(q_counter, q_all_num, q_id)



    # =========================== question 10 ===========================
    # for question 10
    q_id = 10
    q_counter = 0
    q_all_num = len(unique_actions)**2 * len(relation_family)**0 * len(logic_combinator_family)**0 

    for action_1 in unique_actions:
        for action_2 in unique_actions:
            if action_1 == action_2: # avoid meaningless question
                continue
                
            actions = [label_list[0][int(action_1)-1], label_list[0][int(action_2)-1]]
            #action_1_nlp = label_list[0][int(action_1)-1]
            #action_2_nlp = label_list[0][int(action_2)-1]

            q_f_nlp = question_family['questions'][q_id]['texts']
            question_id = random.randint(0,len(q_f_nlp)-1)
            question_nlp = q_f_nlp[question_id]
            question_nlp = fixSentenceTense(question_nlp, actions, [], []) 

            #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
            #question_nlp = question_nlp.replace('<A2>', action_2_nlp)

            # Try generating questions for all possible combinations
            try:
                ans_sm = function_families[q_id](action_1, action_2, 
                                                 scene_lists)
                ans_sm = str(ans_sm)
                ans_nlp = answer_dict[ans_sm]

        #         print(question_nlp, ans_nlp)
                question_family_index.append(q_id)
                question_nl.append(question_nlp)
                answer_nl.append(ans_nlp)
                question_struct.append(str(q_id)+'_'+str(actions))
                
                # Try generating answers for the question
                try:
                    ans_sm_p = function_families[q_id](action_1, action_2, 
                                                 scene_lists_pred, valid_ext = True)
                    ans_nlp_p = answer_dict[str(ans_sm_p)]
                except ValueError:
                    ans_nlp_p = 'Invalid'
                answer_nl_p.append(ans_nlp_p)

                q_counter += 1
            except ValueError:  ####### The question is not valid in the context
                pass   


    if diagnose:
        question_summary(q_counter, q_all_num, q_id) 



    # =========================== question 11 ===========================
    # for question 11
    q_id = 11
    q_counter = 0
    q_all_num = len(unique_actions)**2 * len(relation_family)**0 * len(logic_combinator_family)**0 

    for action_1 in unique_actions:
        for action_2 in unique_actions:
            if action_1 == action_2: # avoid meaningless question
                continue
                
            actions = [label_list[0][int(action_1)-1], label_list[0][int(action_2)-1]]
            #action_1_nlp = label_list[0][int(action_1)-1]
            #action_2_nlp = label_list[0][int(action_2)-1]

            q_f_nlp = question_family['questions'][q_id]['texts']
            question_id = random.randint(0,len(q_f_nlp)-1)
            question_nlp = q_f_nlp[question_id]
            question_nlp = fixSentenceTense(question_nlp, actions, [], []) 

            #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
            #question_nlp = question_nlp.replace('<A2>', action_2_nlp)

            # Try generating questions for all possible combinations
            try:
                ans_sm = function_families[q_id](action_1, action_2, 
                                                 scene_lists)
                ans_sm = str(ans_sm)
                ans_nlp = answer_dict[ans_sm]

        #         print(question_nlp, ans_nlp)
                question_family_index.append(q_id)
                question_nl.append(question_nlp)
                answer_nl.append(ans_nlp)
                question_struct.append(str(q_id)+'_'+str(actions))
                
                # Try generating answers for the question
                try:
                    ans_sm_p = function_families[q_id](action_1, action_2, 
                                                 scene_lists_pred, valid_ext = True)
                    ans_nlp_p = answer_dict[str(ans_sm_p)]
                except ValueError:
                    ans_nlp_p = 'Invalid'
                answer_nl_p.append(ans_nlp_p)

                q_counter += 1
            except ValueError:  ####### The question is not valid in the context
                pass


    if diagnose:
        question_summary(q_counter, q_all_num, q_id)
        
        
    # =========================== question 12 ===========================
    # for question 12
    q_id = 12
    q_counter = 0
    q_all_num = len(unique_actions)**0 * len(relation_family)**0 * len(logic_combinator_family)**0  * len(unique_loc)**1

    for action_1 in unique_loc:

        actions = [label_list[1][int(action_1)-1]]

        q_f_nlp = question_family['questions'][q_id]['texts']
        question_id = random.randint(0,len(q_f_nlp)-1)
        question_nlp = q_f_nlp[question_id]

#         question_nlp = question_nlp.replace('<A1>', action_1_nlp)
        question_nlp = fixSentenceTense(question_nlp, actions, [], []) 

        # Try generating questions for all possible combinations
        try:
            ans_sm = function_families[q_id](action_1, 
                                                scene_lists)
            ans_nlp = label_list[0][int(ans_sm)-1]

    #         print(question_nlp, ans_nlp)
            question_family_index.append(q_id)
            question_nl.append(question_nlp)
            answer_nl.append(ans_nlp)
            question_struct.append(str(q_id)+'_'+str(actions))

            # Try generating answers for the question
            try:
                ans_sm_p = function_families[q_id](action_1, 
                                                scene_lists_pred, valid_ext = True)
                ans_nlp_p = label_list[0][int(ans_sm_p)-1]
            except ValueError:
                ans_nlp_p = 'Invalid'
            answer_nl_p.append(ans_nlp_p)

            q_counter += 1
        except ValueError:  ####### The question is not valid in the context
            pass


    if diagnose:
        question_summary(q_counter, q_all_num, q_id)
        
        
    # =========================== question 13 ===========================
    # for question 13
    q_id = 13
    q_counter = 0
    q_all_num = len(unique_actions)**1 * len(relation_family)**0 * len(logic_combinator_family)**0  * len(unique_loc)**1

    for action_1 in unique_actions:
        for action_2 in unique_loc:

            actions = [label_list[0][int(action_1)-1],label_list[1][int(action_2)-1]]
            #action_1_nlp = label_list[0][int(action_1)-1]
            #action_2_nlp = label_list[1][int(action_2)-1]

            q_f_nlp = question_family['questions'][q_id]['texts']
            question_id = random.randint(0,len(q_f_nlp)-1)
            question_nlp = q_f_nlp[question_id]
            question_nlp = fixSentenceTense(question_nlp, actions, [], []) 

            #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
            #question_nlp = question_nlp.replace('<A2>', action_2_nlp)

            # Try generating questions for all possible combinations
            try:
                ans_sm = function_families[q_id](action_1, action_2, 
                                                    scene_lists,
                                                    question_validation)
                ans_sm = str(ans_sm)
                ans_nlp = ans_sm

        #         print(question_nlp, ans_nlp)
                question_family_index.append(q_id)
                question_nl.append(question_nlp)
                answer_nl.append(ans_nlp)
                question_struct.append(str(q_id)+'_'+str(actions))

                # Try generating answers for the question
                try:
                    ans_sm_p = function_families[q_id](action_1, action_2, 
                                                    scene_lists_pred,
                                                    question_validation = False, valid_ext = True)
                    ans_nlp_p = str(ans_sm_p)
                except ValueError:
                    ans_nlp_p = 'Invalid'
                answer_nl_p.append(ans_nlp_p)

                q_counter += 1
            except ValueError:  ####### The question is not valid in the context
                pass


    if diagnose:
        question_summary(q_counter, q_all_num, q_id)
        
        

    # =========================== question 14 ===========================
    # for question 14
    q_id = 14
    q_counter = 0
    q_all_num = len(unique_actions)**1 * len(relation_family)**0 * len(logic_combinator_family)**0 

    for action_1 in unique_actions:

        actions = [label_list[0][int(action_1)-1]]

        q_f_nlp = question_family['questions'][q_id]['texts']
        question_id = random.randint(0,len(q_f_nlp)-1)
        question_nlp = q_f_nlp[question_id]

        #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
        question_nlp = fixSentenceTense(question_nlp, actions, [], [])

        # Try generating questions for all possible combinations
        try:
            ans_sm = function_families[q_id](action_1, 
                                             scene_lists)
            ans_sm = str(ans_sm)
            ans_nlp = ans_sm

    #         print(question_nlp, ans_nlp)
            question_family_index.append(q_id)
            question_nl.append(question_nlp)
            answer_nl.append(ans_nlp)
            question_struct.append(str(q_id)+'_'+str(actions))
            
            # Try generating answers for the question
            try:
                ans_sm_p = function_families[q_id](action_1, 
                                             scene_lists_pred, valid_ext = True)
                ans_nlp_p = str(ans_sm_p)
            except ValueError:
                ans_nlp_p = 'Invalid'
            answer_nl_p.append(ans_nlp_p)

            q_counter += 1
        except ValueError:  ####### The question is not valid in the context
            pass


    if diagnose:
        question_summary(q_counter, q_all_num, q_id)
        
        
        
    # =========================== question 15 ===========================
    # for question 15
    q_id = 15
    q_counter = 0
    q_all_num = len(unique_actions)**2 * len(relation_family)**1 * len(logic_combinator_family)**0 

    for action_1 in unique_actions:
        for action_2 in unique_actions:
            for relation_1 in relation_family:
                if action_1 == action_2: # avoid meaningless question
                    continue

                #action_1_nlp = label_list[0][int(action_1)-1]
                #action_2_nlp = label_list[0][int(action_2)-1]
                actions = [label_list[0][int(action_1)-1], label_list[0][int(action_2)-1]]
                relations = [relation_1]

                q_f_nlp = question_family['questions'][q_id]['texts']
                question_id = random.randint(0,len(q_f_nlp)-1)
                question_nlp = q_f_nlp[question_id]
                question_nlp = fixSentenceTense(question_nlp, actions, relations, [])

                #question_nlp = question_nlp.replace('<A1>', action_1_nlp)
                #question_nlp = question_nlp.replace('<A2>', action_2_nlp)
                #question_nlp = question_nlp.replace('[R1]', relation_1)

                # Try generating questions for all possible combinations
                try:
                    ans_sm = function_families[q_id](action_1, action_2, 
                                                     relation_1,
                                                     scene_lists,
                                                     question_validation)
                    ans_sm = str(ans_sm)
                    ans_nlp = ans_sm

            #         print(question_nlp, ans_nlp)
                    question_family_index.append(q_id)
                    question_nl.append(question_nlp)
                    answer_nl.append(ans_nlp)
                    question_struct.append(str(q_id)+'_'+str(actions) + '_' + str(relations))
                    
                    # Try generating answers for the question
                    try:
                        ans_sm_p = function_families[q_id](action_1, action_2, 
                                                     relation_1,
                                                     scene_lists_pred,
                                                     question_validation = False, valid_ext = True)
                        ans_nlp_p = str(ans_sm_p)
                    except ValueError:
                        ans_nlp_p = 'Invalid'
                    answer_nl_p.append(ans_nlp_p)

                    q_counter += 1
                except ValueError:  ####### The question is not valid in the context
                    pass


    if diagnose:
        question_summary(q_counter, q_all_num, q_id) 
        
        
    # ====================== Question Generation Finished ================    
    
    return question_family_index, question_nl, answer_nl, answer_nl_p, question_struct

In [12]:
# from sqa_gen.question_generation import question_generator

question_family_file = 'sqa_gen/question_family.json'

question_family_index, question_nl, answer_nl, answer_nl_p, question_struct = question_generator(scene_lists, 
                                                                                                             scene_lists_pred,
                                                                                                             question_family_file,
                                                                                                             label_list,
                                                                                                             show_other = show_other,
                                                                                                             question_validation = True,
                                                                                                             diagnose = True)

Starting generating questions: 
Validation :  True
Size of question families:  17

Unique actions: 
[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
Unique locomotions: 
[2 3 4 5]
**********
['Confirm if the user <A1>?', ['past', 3, 'singular']]
Confirm if the user <A1>?
['Open the front Door']
**********


RuntimeError: generator raised StopIteration